In [1]:
import pandas as pd

anime_data = pd.read_csv('anime.csv')

# synopsis カラムでfloatのものを "" に変換
anime_data['synopsis'] = anime_data['synopsis'].astype(str)

In [2]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import umap
import numpy as np

# Word2Vecモデルの訓練
sentences = [text.split() for text in anime_data['synopsis']]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.train(sentences, total_examples=len(sentences), epochs=100)

# 各文章のベクトルを計算
def vectorize_text(text):
    words = text.split()
    vectors = [model.wv[word] for word in words if word in model.wv.index_to_key]
    if vectors:
        vector = np.mean(vectors, axis=0)
    else:
        vector = np.zeros(model.vector_size) # ベクトルが空の場合、ゼロベクトルを使用
    return vector

vectors = anime_data['synopsis'].apply(vectorize_text)
vectors_matrix = np.vstack(vectors)

# n_neighborsの値をデータセットのサイズよりも小さく設定
n_neighbors_value = min(5, len(anime_data) - 1)

# UMAPで次元削減
umap_model = umap.UMAP(n_neighbors=n_neighbors_value, min_dist=0.3, n_components=10)
umap_vectors = umap_model.fit_transform(vectors_matrix)

# 新規カラムとして追加
anime_data[['umap_vector_' + str(i) for i in range(umap_vectors.shape[1])]] = pd.DataFrame(umap_vectors)


2023-08-23 06:36:43.415519: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-08-23 06:36:43.415557: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# anime.csv として保存
anime_data.to_csv('anime.csv', index=False)